In [1]:
pip install pinecone==6.0.2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from  pinecone import Pinecone

# Get your API key at app.pinecone.io
api_key = os.environ.get("PINECONE_API_KEY")




c:\Users\nikhil.singh\OneDrive - Aionos\Desktop\rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pc = Pinecone(api_key=api_key)

In [6]:
index_name = "swagger-docs"

In [7]:
from pinecone import ServerlessSpec, CloudProvider, AwsRegion, Metric

pc.create_index(
    name=index_name,
    metric=Metric.COSINE,
    dimension=384,
    spec=ServerlessSpec(cloud=CloudProvider.AWS, region=AwsRegion.US_EAST_1),
)

{
    "name": "swagger-docs",
    "metric": "cosine",
    "host": "swagger-docs-r0zzlv1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [8]:
pip install ipykernel

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from langchain_text_splitters import RecursiveJsonSplitter
import json
from langchain.schema import Document
import os
from dotenv import load_dotenv

load_dotenv()

# Import Pinecone
import pinecone

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print(f"Pinecone API Key: {PINECONE_API_KEY}")

# Load and process JSON
with open('./swagger.json', 'r') as f:
    json_data = json.load(f)

# Split the raw JSON
splitter = RecursiveJsonSplitter(max_chunk_size=300)
json_chunks = splitter.split_json(json_data=json_data)

print("First 3 chunks:")
for i in range(0, min(3, len(json_chunks))):
    print(f"Chunk {i}: {json_chunks[i]}")
    print("+++++++")

# Create documents
documents = []
for i, chunk in enumerate(json_chunks):
    text = json.dumps(chunk, indent=2)
    doc = Document(
        page_content=text,
        metadata={
            "chunk_id": i,
            "source": "./swagger.json",
            "keys": list(chunk.keys()) if isinstance(chunk, dict) else []
        }
    )
    documents.append(doc)

print(f"Created {len(documents)} documents")

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("Embeddings model loaded")


Pinecone API Key: pcsk_3sAEUh_DgPkMm6aWnydMM9MtNiSxLiHMXQxWd8xQSjMuixyJ1P5GDnAfhaydrqCzKsfEhB
First 3 chunks:
Chunk 0: {'openapi': '3.0.3', 'info': {'title': 'Aviation Industry - Fake API Collection', 'version': '1.0.0', 'description': 'A large collection (100 endpoints) of fake aviation-related APIs with example responses for testing and demo purposes.'}}
+++++++
Chunk 1: {'servers': [{'url': 'https://api.fake-aviation.example.com', 'description': 'Demo server'}], 'paths': {'/api/v1/aircraft/list': {'get': {'summary': 'List aircraft'}}}}
+++++++
Chunk 2: {'paths': {'/api/v1/aircraft/list': {'get': {'responses': {'200': {'description': 'Array of aircraft', 'content': {'application/json': {'schema': {'type': 'array', 'items': {'$ref': '#/components/schemas/Aircraft'}}, 'examples': {'list': {'$ref': '#/components/examples/AircraftListExample'}}}}}}}}}}
+++++++
Created 185 documents
Embeddings model loaded


In [14]:
description = pc.describe_index(name=index_name)
description

{
    "name": "hello-pinecone",
    "metric": "cosine",
    "host": "hello-pinecone-r0zzlv1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [10]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from langchain_text_splitters import RecursiveJsonSplitter
import json
from langchain.schema import Document
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()

# 1️⃣ Load your text chunks
# chunks = ["This is first chunk", "Second chunk here", "Third one..."]
with open('./swagger.json', 'r') as f:
    json_data = json.load(f)

splitter = RecursiveJsonSplitter(max_chunk_size=300)
json_chunks = splitter.split_json(json_data=json_data)
print(json_chunks)
print("First 3 chunks:")
for i in range(0, min(3, len(json_chunks))):
    print(f"Chunk {i}: {json_chunks[i]}")
    print("+++++++")

# Create documents
documents = []
for i, chunk in enumerate(json_chunks):
    text = json.dumps(chunk, indent=2)
    doc = Document(
        page_content=text,
        metadata={
            "chunk_id": i,
            "source": "./swagger.json",
            "keys": list(chunk.keys()) if isinstance(chunk, dict) else []
        }
    )
    documents.append(doc)
print(documents)

# # 2️⃣ Initialize the embeddings model
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3️⃣ Generate embeddings (each is a list of floats)
embeddings = embed_model.embed_documents(json_chunks)

# print(len(embeddings))          # should be 3
# print(len(embeddings[0]))       # embedding dimension, e.g., 384

# # 4️⃣ Connect to Pinecone
# api_key = os.environ.get("PINECONE_API_KEY")
# pc = Pinecone(api_key=api_key)
# index = pc.Index(host=description.host)

# # 5️⃣ Format data for upsert
# formatted = [
#     {"id": f"chunk-{i}", "values": emb, "metadata": {"text": chunks[i]}}
#     for i, emb in enumerate(embeddings)
# ]

# # 6️⃣ Upsert to Pinecone
# index.upsert(vectors=formatted)
# print("✅ Embeddings successfully inserted!")


[{'openapi': '3.0.3', 'info': {'title': 'Aviation Industry - Fake API Collection', 'version': '1.0.0', 'description': 'A large collection (100 endpoints) of fake aviation-related APIs with example responses for testing and demo purposes.'}}, {'servers': [{'url': 'https://api.fake-aviation.example.com', 'description': 'Demo server'}], 'paths': {'/api/v1/aircraft/list': {'get': {'summary': 'List aircraft'}}}}, {'paths': {'/api/v1/aircraft/list': {'get': {'responses': {'200': {'description': 'Array of aircraft', 'content': {'application/json': {'schema': {'type': 'array', 'items': {'$ref': '#/components/schemas/Aircraft'}}, 'examples': {'list': {'$ref': '#/components/examples/AircraftListExample'}}}}}}}}}}, {'paths': {'/api/v1/aircraft/get': {'get': {'summary': 'Get aircraft by id (query param)', 'parameters': [{'name': 'id', 'in': 'query', 'schema': {'type': 'string'}, 'required': True}]}}}}, {'paths': {'/api/v1/aircraft/get': {'get': {'responses': {'200': {'description': 'Single aircraf

AttributeError: 'dict' object has no attribute 'replace'

In [29]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from langchain_text_splitters import RecursiveJsonSplitter
import json
from langchain.schema import Document
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()

# 1️⃣ Load your text chunks
with open('./swagger.json', 'r') as f:
    json_data = json.load(f)

splitter = RecursiveJsonSplitter(max_chunk_size=300)
json_chunks = splitter.split_json(json_data=json_data)
print(json_chunks)
print("First 3 chunks:")
for i in range(0, min(3, len(json_chunks))):
    print(f"Chunk {i}: {json_chunks[i]}")
    print("+++++++")

# Create documents
documents = []
for i, chunk in enumerate(json_chunks):
    text = json.dumps(chunk, indent=2)
    doc = Document(
        page_content=text,
        metadata={
            "chunk_id": i,
            "source": "./swagger.json",
            "keys": list(chunk.keys()) if isinstance(chunk, dict) else []
        }
    )
    documents.append(doc)
print(documents)

# 2️⃣ Initialize the embeddings model
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3️⃣ Generate embeddings - extract text from documents
# FIXED: Use the page_content from documents instead of raw json_chunks
text_chunks = [doc.page_content for doc in documents]
embeddings = embed_model.embed_documents(text_chunks)

print(f"Number of embeddings: {len(embeddings)}")
print(f"Embedding dimension: {len(embeddings[0])}")

[{'openapi': '3.0.3', 'info': {'title': 'Aviation Industry - Fake API Collection', 'version': '1.0.0', 'description': 'A large collection (100 endpoints) of fake aviation-related APIs with example responses for testing and demo purposes.'}}, {'servers': [{'url': 'https://api.fake-aviation.example.com', 'description': 'Demo server'}], 'paths': {'/api/v1/aircraft/list': {'get': {'summary': 'List aircraft'}}}}, {'paths': {'/api/v1/aircraft/list': {'get': {'responses': {'200': {'description': 'Array of aircraft', 'content': {'application/json': {'schema': {'type': 'array', 'items': {'$ref': '#/components/schemas/Aircraft'}}, 'examples': {'list': {'$ref': '#/components/examples/AircraftListExample'}}}}}}}}}}, {'paths': {'/api/v1/aircraft/get': {'get': {'summary': 'Get aircraft by id (query param)', 'parameters': [{'name': 'id', 'in': 'query', 'schema': {'type': 'string'}, 'required': True}]}}}}, {'paths': {'/api/v1/aircraft/get': {'get': {'responses': {'200': {'description': 'Single aircraf

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import ServerlessSpec
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from langchain_text_splitters import RecursiveJsonSplitter
import json
from langchain.schema import Document
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()

# 1️⃣ Load your text chunks
with open('./swagger.json', 'r') as f:
    json_data = json.load(f)

splitter = RecursiveJsonSplitter(max_chunk_size=300)
json_chunks = splitter.split_json(json_data=json_data)
print(json_chunks)
print("First 3 chunks:")
for i in range(0, min(3, len(json_chunks))):
    print(f"Chunk {i}: {json_chunks[i]}")
    print("+++++++")

# Create documents
documents = []
for i, chunk in enumerate(json_chunks):
    text = json.dumps(chunk, indent=2)
    doc = Document(
        page_content=text,
        metadata={
            "chunk_id": i,
            "source": "./swagger.json",
            "keys": list(chunk.keys()) if isinstance(chunk, dict) else []
        }
    )
    documents.append(doc)
print(documents)

# 2️⃣ Initialize the embeddings model
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3️⃣ Generate embeddings - extract text from documents
text_chunks = [doc.page_content for doc in documents]
embeddings = embed_model.embed_documents(text_chunks)

print(f"Number of embeddings: {len(embeddings)}")
print(f"Embedding dimension: {len(embeddings[0])}")

# 4️⃣ Initialize Pinecone
api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

# Replace 'your-index-name' with your actual index name
index_name = "hello-pinecone"
index = pc.Index(index_name)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1,  # Not used for sparse but required
        metric='dotproduct',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# 5️⃣ Format data for upsert
formatted = [
    {
        "id": f"chunk-{i}", 
        "values": emb, 
        "metadata": {
            "text": text_chunks[i],
            "chunk_id": i,
            "source": "./swagger.json",
            "keys": str(documents[i].metadata.get("keys", []))
        }
    }
    for i, emb in enumerate(embeddings)
]

# 6️⃣ Upsert to Pinecone
index.upsert(vectors=formatted)
print("✅ Embeddings successfully inserted!")
print(f"Total vectors inserted: {len(formatted)}")

[{'openapi': '3.0.3', 'info': {'title': 'Aviation Industry - Fake API Collection', 'version': '1.0.0', 'description': 'A large collection (100 endpoints) of fake aviation-related APIs with example responses for testing and demo purposes.'}}, {'servers': [{'url': 'https://api.fake-aviation.example.com', 'description': 'Demo server'}], 'paths': {'/api/v1/aircraft/list': {'get': {'summary': 'List aircraft'}}}}, {'paths': {'/api/v1/aircraft/list': {'get': {'responses': {'200': {'description': 'Array of aircraft', 'content': {'application/json': {'schema': {'type': 'array', 'items': {'$ref': '#/components/schemas/Aircraft'}}, 'examples': {'list': {'$ref': '#/components/examples/AircraftListExample'}}}}}}}}}}, {'paths': {'/api/v1/aircraft/get': {'get': {'summary': 'Get aircraft by id (query param)', 'parameters': [{'name': 'id', 'in': 'query', 'schema': {'type': 'string'}, 'required': True}]}}}}, {'paths': {'/api/v1/aircraft/get': {'get': {'responses': {'200': {'description': 'Single aircraf

KeyboardInterrupt: 

In [1]:
pip install pinecone-text

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 419.4 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.5 MB 419.4 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.5 MB 419.4 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.5 MB 419.4 kB/s eta 0:00:03
   -------------------- ------------------- 0.8/1.5 MB 372.9 kB/s eta 0:00:02
   -------------------- ------------------- 0.8/1.5 MB 372

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import ServerlessSpec
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from langchain_text_splitters import RecursiveJsonSplitter
import json
from langchain.schema import Document
from pinecone import Pinecone
from pinecone_text.sparse import BM25Encoder
import os
from dotenv import load_dotenv

load_dotenv()

# 1️⃣ Load your text chunks
with open('./swagger.json', 'r') as f:
    json_data = json.load(f)

splitter = RecursiveJsonSplitter(max_chunk_size=300)
json_chunks = splitter.split_json(json_data=json_data)
print(json_chunks)
print("First 3 chunks:")
for i in range(0, min(3, len(json_chunks))):
    print(f"Chunk {i}: {json_chunks[i]}")
    print("+++++++")

# Create documents
documents = []
for i, chunk in enumerate(json_chunks):
    text = json.dumps(chunk, indent=2)
    doc = Document(
        page_content=text,
        metadata={
            "chunk_id": i,
            "source": "./swagger.json",
            "keys": list(chunk.keys()) if isinstance(chunk, dict) else []
        }
    )
    documents.append(doc)
print(documents)

# 2️⃣ Initialize the embeddings model (dense embeddings)
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3️⃣ Generate embeddings - extract text from documents
text_chunks = [doc.page_content for doc in documents]
embeddings = embed_model.embed_documents(text_chunks)

print(f"Number of embeddings: {len(embeddings)}")
print(f"Embedding dimension: {len(embeddings[0])}")

# 3.5️⃣ Initialize BM25 for sparse embeddings
bm25_encoder = BM25Encoder().default()
bm25_encoder.fit(text_chunks)  # Fit BM25 on your corpus
sparse_embeddings = bm25_encoder.encode_documents(text_chunks)

print(f"Number of sparse embeddings: {len(sparse_embeddings)}")

# 4️⃣ Initialize Pinecone
api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

# Replace 'your-index-name' with your actual sparse index name
index_name = "Hybrid Rag"
index = pc.Index(index_name)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1,  # Not used for sparse but required
        metric='dotproduct',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# 5️⃣ Format data for upsert with sparse vectors
formatted = [
    {
        "id": f"chunk-{i}", 
        "sparse_values": {
            "indices": sparse_emb['indices'],
            "values": sparse_emb['values']
        },
        "metadata": {
            "text": text_chunks[i],
            "chunk_id": i,
            "source": "./swagger.json",
            "keys": str(documents[i].metadata.get("keys", []))
        }
    }
    for i, sparse_emb in enumerate(sparse_embeddings)
]

# 6️⃣ Upsert to Pinecone
index.upsert(vectors=formatted)
print("✅ Sparse embeddings successfully inserted!")
print(f"Total vectors inserted: {len(formatted)}")

[{'openapi': '3.0.3', 'info': {'title': 'Aviation Industry - Fake API Collection', 'version': '1.0.0', 'description': 'A large collection (100 endpoints) of fake aviation-related APIs with example responses for testing and demo purposes.'}}, {'servers': [{'url': 'https://api.fake-aviation.example.com', 'description': 'Demo server'}], 'paths': {'/api/v1/aircraft/list': {'get': {'summary': 'List aircraft'}}}}, {'paths': {'/api/v1/aircraft/list': {'get': {'responses': {'200': {'description': 'Array of aircraft', 'content': {'application/json': {'schema': {'type': 'array', 'items': {'$ref': '#/components/schemas/Aircraft'}}, 'examples': {'list': {'$ref': '#/components/examples/AircraftListExample'}}}}}}}}}}, {'paths': {'/api/v1/aircraft/get': {'get': {'summary': 'Get aircraft by id (query param)', 'parameters': [{'name': 'id', 'in': 'query', 'schema': {'type': 'string'}, 'required': True}]}}}}, {'paths': {'/api/v1/aircraft/get': {'get': {'responses': {'200': {'description': 'Single aircraf

100%|██████████| 185/185 [00:00<00:00, 2880.90it/s]


Number of sparse embeddings: 185


NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': 'e4fcd4bdfee27f50b389a6ae9d92a684', 'date': 'Tue, 14 Oct 2025 10:07:59 GMT', 'server': 'Google Frontend', 'Content-Length': '97', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource your-sparse-index-name not found"},"status":404}


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import ServerlessSpec
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from langchain_text_splitters import RecursiveJsonSplitter
import json
from langchain.schema import Document
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()

# 1️⃣ Load your text chunks
with open('./swagger.json', 'r') as f:
    json_data = json.load(f)

splitter = RecursiveJsonSplitter(max_chunk_size=300)
json_chunks = splitter.split_json(json_data=json_data)
print(json_chunks)
print("First 3 chunks:")
for i in range(0, min(3, len(json_chunks))):
    print(f"Chunk {i}: {json_chunks[i]}")
    print("+++++++")

# Create documents
documents = []
for i, chunk in enumerate(json_chunks):
    text = json.dumps(chunk, indent=2)
    doc = Document(
        page_content=text,
        metadata={
            "chunk_id": i,
            "source": "./swagger.json",
            "keys": list(chunk.keys()) if isinstance(chunk, dict) else []
        }
    )
    documents.append(doc)
print(documents)

# 2️⃣ Initialize the embeddings model
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3️⃣ Generate embeddings - extract text from documents
text_chunks = [doc.page_content for doc in documents]
embeddings = embed_model.embed_documents(text_chunks)

print(f"Number of embeddings: {len(embeddings)}")
print(f"Embedding dimension: {len(embeddings[0])}")

# 4️⃣ Initialize Pinecone
api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

# Replace 'your-index-name' with your actual index name
index_name = "hello-pinecone1"

# Create index if it doesn't exist (BEFORE initializing index)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension for all-MiniLM-L6-v2 model
        metric='dotproduct',  # Use cosine for dense embeddings
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    print(f"✅ Created index: {index_name}")

# Now initialize the index
index = pc.Index(index_name)

# 5️⃣ Format data for upsert
formatted = [
    {
        "id": f"chunk-{i}", 
        "values": emb, 
        "metadata": {
            "text": text_chunks[i],
            "chunk_id": i,
            "source": "./swagger.json",
            "keys": str(documents[i].metadata.get("keys", []))
        }
    }
    for i, emb in enumerate(embeddings)
]

# 6️⃣ Upsert to Pinecone
index.upsert(vectors=formatted)
print("✅ Embeddings successfully inserted!")
print(f"Total vectors inserted: {len(formatted)}")

[{'openapi': '3.0.3', 'info': {'title': 'Aviation Industry - Fake API Collection', 'version': '1.0.0', 'description': 'A large collection (100 endpoints) of fake aviation-related APIs with example responses for testing and demo purposes.'}}, {'servers': [{'url': 'https://api.fake-aviation.example.com', 'description': 'Demo server'}], 'paths': {'/api/v1/aircraft/list': {'get': {'summary': 'List aircraft'}}}}, {'paths': {'/api/v1/aircraft/list': {'get': {'responses': {'200': {'description': 'Array of aircraft', 'content': {'application/json': {'schema': {'type': 'array', 'items': {'$ref': '#/components/schemas/Aircraft'}}, 'examples': {'list': {'$ref': '#/components/examples/AircraftListExample'}}}}}}}}}}, {'paths': {'/api/v1/aircraft/get': {'get': {'summary': 'Get aircraft by id (query param)', 'parameters': [{'name': 'id', 'in': 'query', 'schema': {'type': 'string'}, 'required': True}]}}}}, {'paths': {'/api/v1/aircraft/get': {'get': {'responses': {'200': {'description': 'Single aircraf